In [31]:
import winsound
import pandas as pd
from bs4 import BeautifulSoup
import requests
from collections import OrderedDict as od
import pyttsx3


In [59]:
#url = "https://www.benzinga.com/movers"
url = "https://www.benzinga.com/premarket/"
tables = pd.read_html(url)
#df = tables[0]
df = tables[5]
#df.sort_values['Stock']
#stock = df['Stock'].tolist()
sdf1 = df.sort_values('Stock')
sdf1.
#val = sdf1[[0, '% Change']]
#val

,Stock,Company,Price,% Change,Volume
16,ARVL,Arrival,$7.76,5.86%,695.00
13,AVCT,American Virtual Cloud,$1.30,6.55%,447.68K
6,BFRI,Biofrontera,$6.02,9.45%,117.64K
8,BORR,Borr Drilling,$1.65,7.84%,577.00
18,BQ,Boqii Holding,$1.20,5.26%,300.00
9,CFV,CF Acquisition Corp V,$10.57,6.87%,150.00
3,CFVI,CF Acquisition,$12.54,11.36%,249.02K
17,CRBP,Corbus Pharmaceuticals,$0.73,5.79%,25.66K
10,ESSC,East Stone Acquisition,$14.41,6.74%,39.08K
0,GRTX,Galera Therapeutics,$3.14,16.29%,1.46M


In [48]:
#url = "https://www.benzinga.com/movers"
url = "https://www.benzinga.com/premarket/"
tables = pd.read_html(url)

#df = tables[0]
df = tables[5]
#df.sort_values['Stock']
#stock = df['Stock'].tolist()
#https://stackoverflow.com/questions/17095101/compare-two-dataframes-and-output-their-differences-side-by-side
sdf2 = df.sort_values('Stock')
df_all = pd.concat([sdf1[["Stock", "% Change"]].set_index('Stock'), sdf2[["Stock", "% Change"]].set_index('Stock')], 
                   axis='columns', keys=['First', 'Second'])
#df_all['% Diff'] = df_all['First'] - df_all['Second']
df_all = pd.merge(sdf1[['Stock', '% Change']], sdf2[['Stock', '% Change']], how='left', on='Stock')
#df_all['% Diff'] = df_all['% Change_x'] - df_all['% Change_y']
df_all['% Diff'] = df_all['% Change_y'].diff()
df_all

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
duration = 800  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
def diff_func(df1, df2, uid, dedupe=True, labels=('df1', 'df2'), drop=[]):
    dict_df = {labels[0]: df1, labels[1]: df2}
    col1 = df1.columns.values.tolist()
    col2 = df2.columns.values.tolist()

    # There could be columns known to be different, hence allow user to pass this as a list to be dropped.
    if drop:
        print ('Ignoring columns {} in comparison.'.format(', '.join(drop)))
        col1 = list(filter(lambda x: x not in drop, col1))
        col2 = list(filter(lambda x: x not in drop, col2))
        df1 = df1[col1]
        df2 = df2[col2]


    # Step 1 - Check if no. of columns are the same:
    len_lr = len(col1), len(col2)
    assert len_lr[0]==len_lr[1], \
    'Cannot compare frames with different number of columns: {}.'.format(len_lr)

    # Step 2a - Check if the set of column headers are the same
    #           (order doesnt matter)
    assert set(col1)==set(col2), \
    'Left column headers are different from right column headers.' \
       +'\n   Left orphans: {}'.format(list(set(col1)-set(col2))) \
       +'\n   Right orphans: {}'.format(list(set(col2)-set(col1)))

    # Step 2b - Check if the column headers are in the same order
    if col1 != col2:
        print ('[Note] Reordering right Dataframe...')
        df2 = df2[col1]

    # Step 3 - Check datatype are the same [Order is important]
    if set((df1.dtypes == df2.dtypes).tolist()) - {True}:
        print ('dtypes are not the same.')
        df_dtypes = pd.DataFrame({labels[0]:df1.dtypes,labels[1]:df2.dtypes,'Diff':(df1.dtypes == df2.dtypes)})
        df_dtypes = df_dtypes[df_dtypes['Diff']==False][[labels[0],labels[1],'Diff']]
        print (df_dtypes)
    else:
        print ('DataType check: Passed')

    # Step 4 - Check for duplicate rows
    if dedupe:
        for key, df in dict_df.items():
            if df.shape[0] != df.drop_duplicates().shape[0]:
                print(key + ': Duplicates exists, they will be dropped.')
                dict_df[key] = df.drop_duplicates()

    # Step 5 - Check for duplicate uids.
    if type(uid)==str or type(uid)==list:
        print ('Uniqueness check: {}'.format(uid))
        for key, df in dict_df.items():
            count_uid = df.shape[0]
            count_uid_unique = df[uid].drop_duplicates().shape[0]
            var = [0,1][count_uid_unique == df.shape[0]] #<-- Round off to the nearest integer if it is 100%
            pct = round(100*count_uid_unique/df.shape[0], var)
            print ('{}: {} out of {} are unique ({}%).'.format(key, count_uid_unique, count_uid, pct))

    # Checks complete, begin merge. '''Remenber to dedupe, provide labels for common_no_match'''
    dict_result = od()
    df_merge = pd.merge(df1, df2, on=col1, how='inner')
    if not df_merge.shape[0]:
        print ('Error: Merged DataFrame is empty.')
    else:
        dict_result[labels[0]] = df1
        dict_result[labels[1]] = df2
        dict_result['Merge'] = df_merge
        if type(uid)==str:
            uid = [uid]

        if type(uid)==list:
            df1_only = df1.append(df_merge).reset_index(drop=True)
            df1_only['Duplicated']=df1_only.duplicated(keep=False)  #keep=False, marks all duplicates as True
            df1_only = df1_only[df1_only['Duplicated']==False]
            df2_only = df2.append(df_merge).reset_index(drop=True)
            df2_only['Duplicated']=df2_only.duplicated(keep=False)
            df2_only = df2_only[df2_only['Duplicated']==False]

            label = labels[0]+' or '+labels[1]
            df_lc = df1_only.copy()
            df_lc[label] = labels[0]
            df_rc = df2_only.copy()
            df_rc[label] = labels[1]
            df_c = df_lc.append(df_rc).reset_index(drop=True)
            df_c['Duplicated'] = df_c.duplicated(subset=uid, keep=False)
            df_c1 = df_c[df_c['Duplicated']==True]
            df_c1 = df_c1.drop('Duplicated', axis=1)
            df_uc = df_c[df_c['Duplicated']==False]

            df_uc_left = df_uc[df_uc[label]==labels[0]]
            df_uc_right = df_uc[df_uc[label]==labels[1]]

            dict_result[labels[0]+'_only'] = df_uc_left.drop(['Duplicated', label], axis=1)
            dict_result[labels[1]+'_only'] = df_uc_right.drop(['Duplicated', label], axis=1)
            dict_result['Diff'] = df_c1.sort_values(uid).reset_index(drop=True)

    return dict_result

In [ ]:
df1 = pd.DataFrame([['apple', '1'], ['banana', 2], ['coconut',3]], columns=['Fruits','Quantity'])
df2 = pd.DataFrame([['apple', '1'], ['mango', 3], ['durian',4]], columns=['Fruits','Quantity'])
#df2 = pd.DataFrame([['apple', '1'], ['banana', 3], ['durian',4]], columns=['Fruits','Quantity'])
dict1 = diff_func(df1, df2, 'Fruits')
#dict1['df2_only']
dfholder = dict1['df2_only']
dfholder
fruit = dfholder['Fruits'].tolist()
fruit
#bool(dict1['df2_only'].empty)

DataType check: Passed
Uniqueness check: Fruits
df1: 3 out of 3 are unique (100.0%).
df2: 3 out of 3 are unique (100.0%).


['mango', 'durian']

In [ ]:
engine = pyttsx3.init()

#voice_num = 2
text = fruit

#voices = engine.getProperty('voices')
#engine.setProperty('voice', voices[voice_num].id)

engine.say(text)
engine.runAndWait()